In [1]:
from __future__ import print_function
!git clone https://github.com/kundajelab/revcomp_experiments.git
%cd revcomp_experiments/
!python setup.py install
%cd ..

fatal: destination path 'revcomp_experiments' already exists and is not an empty directory.
/mnt/lab_data2/hannahgz/revcomp_experiments/revcomp_experiments/revcomp_experiments
running install
running bdist_egg
running egg_info
writing revcompexp.egg-info/PKG-INFO
writing dependency_links to revcompexp.egg-info/dependency_links.txt
writing requirements to revcompexp.egg-info/requires.txt
writing top-level names to revcompexp.egg-info/top_level.txt
reading manifest file 'revcompexp.egg-info/SOURCES.txt'
writing manifest file 'revcompexp.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib

creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/EGG-INFO
installing scripts to build/bdist.linux-x86_64/egg/EGG-INFO/scripts
running install_scripts
running build_scripts
creating build/bdist.linux-x86_64/egg/EGG-INFO/scripts
copying build/scripts-3.7/motif_density_and_position_sim.py -> build/bdist.linux-x86_64/egg/EGG-INFO/scrip

In [2]:
!pip install concise
!pip install keras-genomics

In [3]:
!pip install git+https://github.com/kundajelab/simdna.git@v0.4.3.1#egg=simdna
!git clone https://github.com/kundajelab/revcomp_experiments.git
%cd revcomp_experiments/
!python setup.py install
%cd ..

fatal: destination path 'revcomp_experiments' already exists and is not an empty directory.
/mnt/lab_data2/hannahgz/revcomp_experiments/revcomp_experiments/revcomp_experiments
running install
running bdist_egg
running egg_info
writing revcompexp.egg-info/PKG-INFO
writing dependency_links to revcompexp.egg-info/dependency_links.txt
writing requirements to revcompexp.egg-info/requires.txt
writing top-level names to revcompexp.egg-info/top_level.txt
reading manifest file 'revcompexp.egg-info/SOURCES.txt'
writing manifest file 'revcompexp.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib

creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/EGG-INFO
installing scripts to build/bdist.linux-x86_64/egg/EGG-INFO/scripts
running install_scripts
running build_scripts
creating build/bdist.linux-x86_64/egg/EGG-INFO/scripts
copying build/scripts-3.7/motif_density_and_position_sim.py -> build/bdist.linux-x86_64/egg/EGG-INFO/scrip

In [4]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [5]:
!motif_density_and_position_sim.py --seed 1234 --motifNames GATA_disc1 TAL1_known1 --max-motifs 4 --min-motifs 1 --mean-motifs 2 --zero-prob 0.5 --seqLength 1000 --posSdevInBp 100 --numSeqs 10000

In [6]:
!head DensityEmbedding_motifs-GATA_disc1+TAL1_known1_min-1_max-4_mean-2_zeroProb-0p5_seqLength-1000_posSdevInBp-100_numSeqs-10000.simdata

seqName	sequence	embeddings
synth0	AGCTTAATTTCGGTCGGATTCGACTGCTCGTCTATTATCTAAAGGGAGCGAGGAGTTTTTAGCACACTAATGCAATCGACCGTGTATTGCAGGTCGGTAGTTTGTAAGATCACTTACATTTTTGAACGAACGTAAATACGATTGACTAGATGAATTTTGGAGTTTTGCTTGCCTATCTTTGCTTCACTCCGAATACGATTTTGTTTACACATGGGCCCCTCATTCGCAACAGTTCACTTCAAAGAAAGATCTGAGGATGGGGATTTGCTGCGCAGTTGACAAGAGGTTTAAGTTGCGTGAATCGGAAGTCTAAAAAGTTGTGGGTATAAAAGCTACTCTGTTTATGATAATTGCAGTTTAGCCATGATTCTGTTTACCATCTGTACACTGTAACTTCTACTTAAGTCAGGATTTATTTTACTTGTGATCTCATAGAAGTGTCAATACAGGGTTATGATGGCCTAACAGATGTAATAAACGATATCTCACTAGCGAATTAGTTGAATAGTAGCAAAAACATTTAGTTACGGGTTCATCATAGCAACAGATGGCCGAATTCAAATACGGCCACTTATATAACTTCAGAAGGATTTCCCCGGATTATAGAACACTGGGAGTAGTTCCTAAACAGACCTGGGATCAGATGGTCGACTTCAGCTGTTCAACGGGATTCTAAGTAGAAATCAGGCTACACACGAAATCCTACGTCTGTTTAAAATTTCGAATATTCTCAGCGCGTGTAAGGTTTGATCGGCACTGAGATTGCCTAGCAATTTCAGGTGATTATATTTGCGATACTCGATGCATAAAGGTATACTAATTAGCTAAACAAAATATCTACGGCCCCCCGTGCATAGTTCATGTGCTTGTATTCTGCGTACTGCACTGCGATGTTCTTGGTCATGCATGACGCCCGTTTGGTGTTTGACAATAAGGCCAAAATCTTTTGCTAACCTGAGTATATG

In [9]:
#@title
from __future__ import absolute_import, division, print_function
from collections import OrderedDict
import re
import gzip
import os
import json
from simdna import random


DEFAULT_LETTER_TO_INDEX = {'A': 0, 'C': 1, 'G': 2, 'T': 3}


DEFAULT_BACKGROUND_FREQ = OrderedDict(
    [('A', 0.3), ('C', 0.2), ('G', 0.2), ('T', 0.3)])


DEFAULT_DINUC_FREQ = OrderedDict([
 ('AA',0.095),
 ('AC',0.050),
 ('AG',0.071),
 ('AT',0.075),
 ('CA',0.073),
 ('CC',0.054),
 ('CG',0.010),
 ('CT',0.072),
 ('GA',0.060),
 ('GC',0.044),
 ('GG',0.054),
 ('GT',0.050),
 ('TA',0.064),
 ('TC',0.060),
 ('TG',0.073),
 ('TT',0.095),
])


def get_file_handle(filename,mode="r"):
    if (re.search('.gz$',filename) or re.search('.gzip',filename)):
        if (mode=="r"):
            mode="rb";
        elif (mode=="w"):
            #I think write will actually append if the file already
            #exists...so you want to remove it if it exists
            if os.path.isfile(filename):
                os.remove(filename);
        return gzip.open(filename,mode)
    else:
        return open(filename,mode) 


def default_tab_seppd(s):
    s = trim_newline(s)
    s = split_by_delimiter(s, "\t")
    return s


def trim_newline(s):
    return s.rstrip('\r\n')


def split_by_delimiter(s, delimiter):
    return s.split(delimiter)


def perform_action_on_each_line_of_file(
    file_handle
    #should be a function that accepts the
    #preprocessed/filtered line and the line number
    , action
    , transformation=default_tab_seppd
    , ignore_input_title=False
    , progress_update=None
    , progress_update_file_name=None):

    i = 0;
    for line in file_handle:
        i += 1;
        line = line.decode("utf-8")
        process_line(line, i, ignore_input_title,
                     transformation, action, progress_update)
        print_progress(progress_update, i, progress_update_file_name)

    file_handle.close();


def process_line(line, i, ignore_input_title,
                 transformation, action, progress_update=None):
    if (i > 1 or (ignore_input_title==False)):
        action(transformation(line),i)


def print_progress(progress_update, i, file_name=None):
    if progress_update is not None:
        if (i%progress_update == 0):
            print ("Processed "+str(i)+" lines"
                   +str("" if file_name is None else " of "+file_name))


class VariableWrapper():
    """ For when I want reference-type access to an immutable"""
    def __init__(self, var):
        self.var = var   


def enum(**enums):
    class Enum(object):
        pass
    to_return = Enum
    for key,val in enums.items():
        if hasattr(val, '__call__'): 
            setattr(to_return, key, staticmethod(val))
        else:
            setattr(to_return, key, val)
    to_return.vals = [x for x in enums.values()]
    to_return.the_dict = enums
    return to_return


def combine_enums(*enums):
    new_enum_dict = OrderedDict()
    for an_enum in enums:
        new_enum_dict.update(an_enum.the_dict)
    return enum(**new_enum_dict)


def sampleFromProbsArr(arrWithProbs):
    """Samples from a discrete distribution.
    Arguments:
        arrWithProbs: array of probabilities
    Returns:
        an index, sampled with the probability of that index in
    array of probabilities.
    """
    randNum = random.random()
    cdfSoFar = 0
    for (idx, prob) in enumerate(arrWithProbs):
        cdfSoFar += prob
        if (cdfSoFar >= randNum or idx == (len(arrWithProbs) - 1)):  # need the
            # letterIdx==(len(row)-1) clause because of potential floating point errors
            # that mean arrWithProbs doesn't sum to 1
            return idx


reverseComplementLookup = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G',
                           'a': 't', 't': 'a', 'g': 'c', 'c': 'g', 'N': 'N', 'n': 'n'}


def reverseComplement(sequence):
    reversedSequence = sequence[::-1]
    reverseComplemented = "".join(
        [reverseComplementLookup[x] for x in reversedSequence])
    return reverseComplemented


def sampleWithoutReplacement(arr, numToSample):
    arrayCopy = [x for x in arr]
    for i in range(numToSample):
        randomIndex = int(random.random() * (len(arrayCopy) - i)) + i
        swapIndices(arrayCopy, i, randomIndex)
    return arrayCopy[0:numToSample]


def swapIndices(arr, idx1, idx2):
    temp = arr[idx1]
    arr[idx1] = arr[idx2]
    arr[idx2] = temp


def get_file_name_parts(file_name):
    p = re.compile(r"^(.*/)?([^\./]+)(\.[^/]*)?$")
    m = p.search(file_name)
    return FileNameParts(m.group(1), m.group(2), m.group(3))


class FileNameParts(object):

    def __init__(self, directory, core_file_name, extension):
        self.directory = directory if (directory is not None) else os.getcwd()
        self.core_file_name = core_file_name
        self.extension = extension

    def get_full_path(self):
        return self.directory+"/"+self.file_name

    def get_core_file_name_and_extension(self):
        return self.core_file_name+self.extension

    def get_transformed_core_file_name(self, transformation, extension=None):
        to_return = transformation(self.core_file_name)
        if (extension is not None):
            to_return = to_return + extension
        else:
            if (self.extension is not None):
                to_return = to_return + self.extension
        return to_return

    def get_transformed_file_path(self, transformation, extension=None):
        return (self.directory+"/"+
                self.get_transformed_core_file_name(transformation,
                                                    extension=extension))


def format_as_json(jsonable_data):
    return json.dumps(jsonable_data, indent=4, separators=(',', ': '))


class ArgumentToAdd(object):
    """
        Class to append runtime arguments to a string
        to facilitate auto-generation of output file names.
    """
    def __init__(self, val, argumentName=None, argNameAndValSep="-"):
        self.val = val;
        self.argumentName = argumentName;
        self.argNameAndValSep = argNameAndValSep;
    def argNamePrefix(self):
        return ("" if self.argumentName is None else self.argumentName+str(self.argNameAndValSep))
    def transform(self):
        string = (','.join([str(el) for el in self.val])\
                   if (isinstance(self.val, str)==False and
                       hasattr(self.val,"__len__")) else str(self.val))
        return self.argNamePrefix()+string;
        # return self.argNamePrefix()+str(self.val).replace(".","p");


class FloatArgument(ArgumentToAdd):
    """
       Replace the period with a p 
    """
    def __init__(self, val, argumentName=None, argNameAndValSep="-"):
        self.val = val;
        self.argumentName = argumentName;
        self.argNameAndValSep = argNameAndValSep;
    def argNamePrefix(self):
        return ("" if self.argumentName is None else self.argumentName+str(self.argNameAndValSep))
    def transform(self):
        string = str(self.val)
        string = string.replace(".","p")
        return self.argNamePrefix()+string


class BooleanArgument(ArgumentToAdd):

    def transform(self):
        assert self.val  # should be True if you're calling transformation
        return self.argumentName


class CoreFileNameArgument(ArgumentToAdd):

    def transform(self):
        import fileProcessing as fp
        return self.argNamePrefix() + fp.getCoreFileName(self.val)


class ArrArgument(ArgumentToAdd):

    def __init__(self, val, argumentName, sep="+", toStringFunc=str):
        super(ArrArgument, self).__init__(val, argumentName)
        self.sep = sep
        self.toStringFunc = toStringFunc

    def transform(self):
        return self.argNamePrefix() + self.sep.join([self.toStringFunc(x) for x in self.val])


class ArrOfFileNamesArgument(ArrArgument):

    def __init__(self, val, argumentName, sep="+"):
        import fileProcessing as fp
        super(ArrOfFileNamesArgument, self).__init__(val, argumentName,
                                                     sep, toStringFunc=lambda x: fp.getCoreFileName(x))


def addArguments(string, args, joiner="_"):
    """
        args is an array of ArgumentToAdd.
    """
    for arg in args:
        string = string + ("" if arg.val is None or arg.val is False or (hasattr(
            arg.val, "__len__") and len(arg.val) == 0) else joiner + arg.transform())
    return string
class Embedding(object):
    """Represents something that has been embedded in a sequence.
    Think of this as a combination of an embeddable + a start position.
    Arguments:
        what: object representing the thing that has been embedded.\
            Should have`` __str__`` and ``__len__`` defined.\
            Often is an instance of :class:`.AbstractEmbeddable`
        startPos: int, the position relative to the start of the parent\
            sequence at which seq has been embedded
    """

    def __init__(self, what, startPos):
        self.what = what
        self.startPos = startPos

    def __str__(self):
        return "pos-" + str(self.startPos) + "_" + str(self.what)

    @classmethod
    def fromString(cls, string, whatClass=None):
        """Recreate an :class:`.Embedding` object from a string.
        Arguments:
            string: assumed to have format:\
                ``description[-|_]startPos[-|_]whatString``, where
                ``whatString`` will be provided to ``whatClass``
            whatClass: the class (usually a :class:`.AbstractEmbeddable`) that\
                will be used to instantiate the what from the whatString
        Returns:
            The Embedding class called with
            ``what=whatClass.fromString(whatString)`` and
            ``startPos=int(startPos)``
        """
        if (whatClass is None):
            from simdna.synthetic.embeddables import StringEmbeddable
            whatClass = StringEmbeddable
        # was printed out as pos-[startPos]_[what], but the
        #[what] may contain underscores, hence the maxsplit
        # to avoid splitting on them.
        p = re.compile(r"pos\-(\d+)_(.*)$")
        m = p.search(string)
        startPos = m.group(1)
        whatString = m.group(2) 
        return cls(what=whatClass.fromString(whatString),
                   startPos=int(startPos))
      
def getEmbeddingsFromString(string):
    """Get a series of :class:`.Embedding` objects from a string.
    
    Splits the string on commas, and then passes the comma-separated vals
        to :func:`.Embedding.fromString`
    Arguments:
        string: The string to turn into an array of Embedding objects
    Returns:
        an array of :class:`.Embedding` objects
    """
    if len(string) == 0:
        return []
    else:
        embeddingStrings = string.split(",")
        return [Embedding.fromString(x) for x in embeddingStrings]
      
def read_simdata_file(simdata_file, one_hot_encode=False, ids_to_load=None):
    ids = []
    sequences = []
    embeddings = []
    labels = []
    if (ids_to_load is not None):
        ids_to_load = set(ids_to_load)
    def action(inp, line_number):
        if (line_number > 1):
            if (ids_to_load is None or (inp[0] in ids_to_load)):
                ids.append(inp[0]) 
                sequences.append(inp[1])
                embeddings.append(getEmbeddingsFromString(inp[2]))
                labels.append([int(x) for x in inp[3:]])
    perform_action_on_each_line_of_file(
        file_handle=get_file_handle(simdata_file),
        action=action,
        transformation=default_tab_seppd)
    return enum(
            ids=ids,
            sequences=sequences,
            embeddings=embeddings,
            labels=np.array(labels))
  


def perform_action_on_each_line_of_file(
    file_handle
    #should be a function that accepts the
    #preprocessed/filtered line and the line number
    , action
    , transformation=default_tab_seppd
    , ignore_input_title=False
    , progress_update=None
    , progress_update_file_name=None):

    i = 0;
    for line in file_handle:
        i += 1;
#         line = line.decode("utf-8")
        process_line(line, i, ignore_input_title,
                     transformation, action, progress_update)
        print_progress(progress_update, i, progress_update_file_name)

    file_handle.close();

In [10]:
import keras 
import keras_genomics
import numpy as np
import simdna

from keras import backend as K 
from keras.layers.core import Dropout 
from keras.layers.core import Flatten
from keras.engine import Layer
from keras.models import Sequential 
import keras.layers as k1
from keras.engine.base_layer import InputSpec

# from simdna.synthetic.core import read_simdata_file

from sklearn.model_selection import train_test_split

In [11]:
sequences = read_simdata_file("DensityEmbedding_motifs-GATA_disc1+TAL1_known1_min-1_max-4_mean-2_zeroProb-0p5_seqLength-1000_posSdevInBp-100_numSeqs-10000.simdata").sequences
embeddings = read_simdata_file("DensityEmbedding_motifs-GATA_disc1+TAL1_known1_min-1_max-4_mean-2_zeroProb-0p5_seqLength-1000_posSdevInBp-100_numSeqs-10000.simdata").embeddings

In [12]:
def check(embeddings): 
    if len(embeddings) > 0:
        all_chars = ""
        for x in embeddings: 
            all_chars += str(x.what)
        if 'TAL' in all_chars and 'GATA' in all_chars: 
            return [1, 1]
        if 'TAL' in all_chars and 'GATA' not in all_chars:
            return [1, 0]
        if 'TAL' not in all_chars and 'GATA' in all_chars:
            return [0, 1]
    return [0,0]

In [13]:
y= np.zeros((0,2))
for x in embeddings: 
    thing = check(x)
    y = np.append(y, [thing], axis=0)

In [14]:
ltrdict = {'a':[1,0,0,0],
           'c':[0,1,0,0],
           'g':[0,0,1,0],
           't':[0,0,0,1],
           'n':[0,0,0,0],
           'A':[1,0,0,0],
           'C':[0,1,0,0],
           'G':[0,0,1,0],
           'T':[0,0,0,1],
           'N':[0,0,0,0]}

def onehot_encode(sequence):
    return np.array([ltrdict[x] for x in sequence])

x = np.array([onehot_encode(x) for x in sequences]) 

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [17]:
x_train[0].shape

(1000, 4)

In [18]:
kernel_size = 15
filters=15
input_length = 1000

In [38]:
seed_num = 1000

from numpy.random import seed
from tensorflow import set_random_seed
from keras.callbacks import EarlyStopping, History

seed(seed_num)
set_random_seed(seed_num)

In [20]:
from keras.layers import Input
from keras.models import Model
from keras.models import load_model

In [39]:
filename = ('no_batch_standard_%s.txt' % seed_num, str(seed_num))[0]
f = open(filename, 'w')

In [30]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

def evaluate(model, x, y): 
    y_pred = model.predict(x)
    auroc = roc_auc_score(y, model.predict(x)) 
    auprc = average_precision_score(y, model.predict(x))
    print("auroc: " + str(auroc))
    print("auprc: "+ str(auprc))
    f.write("auroc: " + str(auroc) + "\n")
    f.write("auprc: " + str(auprc) + "\n")

# Computing Gradients

In [ ]:
import statistics
def get_session():
    try:
        #use the keras session if there is one
        import keras.backend as K
        return K.get_session()
    except:
        #Warning: I haven't really tested this behaviour out...
        global _SESS 
        if _SESS is None:
            print("MAKING A SESSION")
            _SESS = tf.Session()
            _SESS.run(tf.global_variables_initializer()) 
        return _SESS

def compile_func(inputs, outputs):
    if (isinstance(inputs, list)==False):
        print("Wrapping the inputs in a list...")
        inputs = [inputs]
    assert isinstance(inputs, list)
    def func_to_return(inp):
        if len(inp) > len(inputs) and len(inputs)==1:
            print("Wrapping the inputs in a list...")
            inp = [inp]
        assert len(inp)==len(inputs),\
            ("length of provided list should be "
             +str(len(inputs))+" for tensors "+str(inputs)
             +" but got input of length "+str(len(inp)))
        feed_dict = {}
        for input_tensor, input_val in zip(inputs, inp):
            feed_dict[input_tensor] = input_val 
        sess = get_session()
        return sess.run(outputs, feed_dict=feed_dict)  
    return func_to_return

import tensorflow as tf
# grads_tensor = tf.gradients(ys=model.layers[-1].output, xs=model.input)
# grad_func = compile_func(inputs=model.input, outputs=mode.output)
# gradients = grad_func(batch_of_one_hot_encoded_input)

# Reverse Complement

In [ ]:
rc_model = keras.models.Sequential()
rc_model.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, 
            input_shape=x_train.shape[1:], padding="same", 
            kernel_initializer = "he_normal"))
rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
rc_model.add(k1.core.Activation("relu"))
rc_model.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, padding="same", 
            kernel_initializer = "he_normal"))
rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
rc_model.add(k1.core.Activation("relu"))
rc_model.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size,padding="same", 
            kernel_initializer = "he_normal"))
rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
rc_model.add(k1.core.Activation("relu"))
rc_model.add(RevCompSumPool())
rc_model.add(k1.pooling.MaxPooling1D(pool_size=20,padding="same", strides=20))
rc_model.add(Flatten())
rc_model.add(keras_genomics.layers.core.Dense(units = 100, activation = "relu", 
                                             kernel_initializer = "he_normal"))
rc_model.add(keras_genomics.layers.core.Dense(units = 2))
rc_model.add(k1.core.Activation("sigmoid"))

In [ ]:
grads_tensor_rc = tf.gradients(ys = rc_model.layers[-1].output, xs = rc_model.input)
grad_fun = compile_func(inputs = rc_model.input, outputs = grads_tensor_rc)
gradients = grad_fun(x_train[0:500])

In [ ]:
sum = 0
for x in gradients[0]:
    for y in x: 
        for z in y: 
            sum+=z
print(z/2000000)
print("RevComp he_normal")

In [ ]:
rc_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history_rc = rc_model.fit(x_train, y_train, validation_split=0.2,  
                    callbacks=[EarlyStopping(patience=10, restore_best_weights=True),  
                               History()], batch_size=100,  epochs=100) 

In [ ]:
rc_filename = ('batch_only_%s.h5' % seed_num, str(seed_num))[0]
rc_model.save(rc_filename)
custom_objects = {'RevCompConv1D':keras_genomics.layers.RevCompConv1D, 
                  'RevCompConv1DBatchNorm':keras_genomics.layers.RevCompConv1DBatchNorm,
                  'RevCompSumPool':RevCompSumPool}
rc_model_final = load_model(rc_filename, custom_objects)

# Regular Model

In [40]:
reg_model = keras.models.Sequential()
reg_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        input_shape=(input_length,4), padding="same"))
reg_model.add(k1.BatchNormalization())
reg_model.add(k1.core.Activation("relu"))
reg_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        padding="same"))
reg_model.add(k1.BatchNormalization())
reg_model.add(k1.core.Activation("relu"))
reg_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        padding="same"))
reg_model.add(k1.BatchNormalization())
reg_model.add(k1.core.Activation("relu"))
reg_model.add(k1.pooling.MaxPooling1D(pool_size=20,padding="same",
                                               strides=20))
reg_model.add(Flatten())
reg_model.add(k1.Dense(units = 100, activation = "relu"))
reg_model.add(k1.Dense(units = 2))
reg_model.add(k1.core.Activation("sigmoid"))

In [41]:
reg_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history_reg = reg_model.fit(x_train, y_train, validation_split = 0.2, 
                           callbacks=[EarlyStopping(patience=10,restore_best_weights=True), 
                                      History()], batch_size = 100, epochs = 100)

Train on 6400 samples, validate on 1600 samples
Epoch 1/100
6400/6400 [==============================] - 3s 468us/step - loss: 0.6956 - acc: 0.5064 - val_loss: 0.6931 - val_acc: 0.5041
Epoch 2/100
6400/6400 [==============================] - 2s 345us/step - loss: 0.6923 - acc: 0.5127 - val_loss: 0.6930 - val_acc: 0.5031
Epoch 3/100
6400/6400 [==============================] - 2s 349us/step - loss: 0.6914 - acc: 0.5290 - val_loss: 0.6940 - val_acc: 0.4912
Epoch 4/100
6400/6400 [==============================] - 2s 355us/step - loss: 0.6872 - acc: 0.5509 - val_loss: 0.6964 - val_acc: 0.4897
Epoch 5/100
6400/6400 [==============================] - 2s 334us/step - loss: 0.6787 - acc: 0.5730 - val_loss: 0.6997 - val_acc: 0.5269
Epoch 6/100
6400/6400 [==============================] - 2s 344us/step - loss: 0.6423 - acc: 0.6384 - val_loss: 0.6443 - val_acc: 0.6109
Epoch 7/100
6400/6400 [==============================] - 2s 340us/step - loss: 0.4966 - acc: 0.7530 - val_loss: 0.4832 - val_acc: 

In [42]:
reg_filename = ('reg_no_batch_standard_%s.h5' % seed_num, str(seed_num))[0]
reg_model.save(reg_filename)
reg_model_final = load_model(reg_filename)

In [43]:
print("Regular Model")
f.write("Regular Model\n")
evaluate(reg_model, x_test, y_test)

Regular Model
auroc: 0.9404323818999469
auprc: 0.9504508169739929


In [44]:
f.close()

# Siamese Architecture

In [ ]:
s_model = Sequential([
    k1.Conv1D(filters=filters, kernel_size=kernel_size,
            input_shape=(input_length,4), padding="same"), 
    k1.BatchNormalization(), 
    k1.core.Activation("relu"),
    k1.Conv1D(filters=filters, kernel_size=kernel_size,
              padding="same"), 
    k1.BatchNormalization(), 
    k1.core.Activation("relu"),
    k1.Conv1D(filters=filters, kernel_size=kernel_size,
              padding="same"), 
    k1.BatchNormalization(), 
    k1.core.Activation("relu"),
    k1.pooling.MaxPooling1D(pool_size=20,padding="same",
                                               strides=20), 
    Flatten(), 
    k1.Dense(units = 100, activation = "relu"),
    k1.Dense(units = 2)
], name = "shared_layers")

main_input = Input(shape=(input_length, 4, ))
rev_input = Input(shape=(input_length, 4, ))

main_output = s_model(main_input)
rev_output = s_model(rev_input)

avg = k1.Average()([main_output, rev_output])
final_out = k1.core.Activation("sigmoid")(avg)
siamese_model = Model(inputs = [main_input, rev_input], outputs = final_out)

merged = keras.layers.concatenate([main_output, rev_output])

predictions = k1.core.Activation("sigmoid")

In [ ]:
siamese_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
siamese_model.fit([x_train, x_train[:,::-1,::-1]], y_train, validation_split=0.2,  
                    callbacks=[EarlyStopping(patience=10, restore_best_weights=True), History()],
                    batch_size=100,  epochs=100)

In [ ]:
siamese_filename = ('batch_only_siamese_%s.h5' % seed_num, str(seed_num))[0]
siamese_model.save(siamese_filename)
siamese_model_final = load_model(siamese_filename)

# Evaluating Models

In [ ]:
print("Reverse Complement Model")
f.write("Reverse Complement Model\n")
evaluate(rc_model, x_test, y_test)

In [ ]:
print("Regular Model")
f.write("Regular Model\n")
evaluate(reg_model, x_test, y_test)

In [ ]:
print("Siamese Architecture")
y_pred = siamese_model.predict([x_test, x_test[:,::-1,::-1]])
auroc = roc_auc_score(y_test, y_pred)
auprc = average_precision_score(y_test, y_pred)
print("auroc: " + str(auroc))
print("auprc: "+ str(auprc))
f.write("Siamese Architecture\n")
f.write("auroc: " + str(auroc) + "\n")
f.write("auprc: "+ str(auprc) + "\n")

In [ ]:
f.close()